# 1. Introducing Optim.jl
This notebook introduces the very basic features of Optim.jl, a package for non-linear optimization in Julia. We focus on syntax in this tutorial. More advanced topics are discussed in other notebooks. First of all, we need to install and load Optim.

In [2]:
Pkg.add("Optim")
using Optim

INFO: Package Optim is already installed
INFO: METADATA is out-of-date — you may not have the latest version of Optim
INFO: Use `Pkg.update()` to get the latest versions of your packages


Installing Optim will also install a few other packages, the so-called dependencies. Most of these are not of special interest to you as the end-user, but one is not like the others: LineSearches.jl. We will see later how to use this, and in another notebook we will look closer at the different methods and options related this package.

In a nutshell, Optim is all about optimizing a function $f:\mathbb{R}^N\to\mathbb{R}$ (or by proper transformation also $f:\mathbb{C}^N\to\mathbb{R}$). By optimizing we mean finding an input that makes the function value as small as possible. This does not always come without complications, as we will see below. First, let us consider a specific example: the Himmelblau test function. It is named after David Mautner Himmelblau (1924-2011) who introduced it in his *Applied Nonlinear Programming (1972)* textbook. Mathematically, define
$$
f(x)=(x_1^{2}+x_2-11)^{2}+(x_1+x_2^{2}-7)^{2}
$$
In code, we get the following, that also includes the gradient and Hessian as they will come in handy later.

In [3]:
function himmelblau(x)
    return (x[1]^2 + x[2] - 11)^2 + (x[1] + x[2]^2 - 7)^2
end

function himmelblau_gradient!(storage, x)
    storage[1] = 4.0 * x[1]^3 + 4.0 * x[1] * x[2] -
                  44.0 * x[1] + 2.0 * x[1] + 2.0 * x[2]^2 - 14.0
    storage[2] = 2.0 * x[1]^2 + 2.0 * x[2] - 22.0 +
                  4.0 * x[1] * x[2] + 4.0 * x[2]^3 - 28.0 * x[2]
end

function himmelblau_hessian!(storage, x)
    storage[1, 1] = 12.0 * x[1]^2 + 4.0 * x[2] - 42.0
    storage[1, 2] = 4.0 * x[1] + 4.0 * x[2]
    storage[2, 1] = 4.0 * x[1] + 4.0 * x[2]
    storage[2, 2] = 12.0 * x[2]^2 + 4.0 * x[1] - 26.0
end

himmelblau_hessian! (generic function with 1 method)

To get an idea about the problem we're facing, let us make a plot that shows what choices of $x$ gives high and low values of the objective function.

In [4]:
using Plots
gr()
x = -5.0:0.2:5.0
y = -5.0:0.2:5.0
X = repmat(x',length(y),1)
Y = repmat(y,1,length(x))
p = contour(x,y,(x,y)->himmelblau([x,y]),fill=true, color=:deep)
contour!(x,y,(x,y)->himmelblau([x,y]))
scatter!([3.0, -2.805118, -3.779310, 3.584428], [2.0, 3.131312, -3.283186, -1.848126], legend=false, color=:black, colorbar=false)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -4 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 -4 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 
 
<polyline clip-path="url(#clip1102)" style="stroke:#470b69; stroke-width:1; stroke-opacity:1; fill:none" points="
 45.0562,355.526 50.9731,358.696 55.4496,360.52 65.8429,363.284 76.2363,364.774 86.6297,365.23 97.0231,364.853 107.416,363.814 117.81,362.259 128.203,360.312 
 135.677,358.696 138.597,357.955 148.99,355.051 159.383,352.028 160.051,351.827 169.777,348.371 179.507,344.958 180.17,344.681 190.563,340.396 196.43,338.089 
 200.957,335.949 211.35,331.348 211.66,331.22 221.744,326.174 225.794,324.351 232.137,320.837 239.042,317.482 242.53,315.359 251.707,310.613 252.924,309.809 
 263.317,304.191 264.378,303.744 273.71,298.643 278.322,296.875 284.104,293.912 294.497,290.799 301.685,290.006 304.891,289.523 311.062,290.006 315.284,290.224 
 325.677,292.216 336.071,295.45 339.431,296.875 346.464,299.047 356.857,303.018 358.47,303.744 367.251,306.765 377.224,310.613 377.644,310.741 388.038,314.217 
 397.26,317.482 398.431,317.816 408.824,320.911 419.218,324.03 420.302,324.351 429.611,326.623 440.004,329.041 450.216,331.22 450.398,331.252 460.791,332.874 
 471.185,334.176 481.578,335.074 491.971,335.474 502.365,335.28 512.758,334.384 523.151,332.672 528.842,331.22 533.545,329.785 543.938,325.323 545.644,324.351 
 554.332,318.311 555.264,317.482 561.459,310.613 564.725,305.952 
 "/>
<polyline clip-path="url(#clip1102)" style="stroke:#63146e; stroke-width:1; stroke-opacity:1; fill:none" points="
 564.725,323.336 563.584,324.351 554.332,331.074 554.074,331.22 543.938,336.024 538.16,338.089 533.545,339.484 523.151,341.727 512.758,343.181 502.365,343.949 
 491.971,344.128 481.578,343.807 471.185,343.07 460.791,341.995 450.398,340.654 440.004,339.115 433.62,338.089 429.611,337.326 419.218,335.263 408.824,333.173 
 398.955,331.22 398.431,331.095 388.038,328.743 377.644,326.552 367.251,324.578 365.855,324.351 356.857,322.556 346.464,320.866 336.071,319.611 325.677,318.845 
 315.284,318.618 304.891,318.979 294.497,319.976 284.104,321.654 273.71,324.058 272.726,324.351 263.317,326.658 252.924,329.823 249.038,331.22 242.53,333.169 
 232.137,336.754 228.675,338.089 221.744,340.334 211.35,344.042 208.954,344.958 200.957,347.545 190.563,351.121 188.575,351.827 180.17,354.372 169.777,357.6 
 166.237,358.696 159.383,360.516 148.99,363.219 139.583,365.565 
 "/>
<polyline clip-path="url(#clip1102)" style="stroke:#821f6b; stroke-width:1; stroke-opacity:1; fill:none" points="
 167.824,365.565 169.777,365.031 180.17,362.167 190.563,359.376 193.169,358.696 200.957,356.34 211.35,353.366 217.103,351.827 221.744,350.379 232.137,347.4 
 241.656,344.958 242.53,344.695 252.924,341.912 263.317,339.544 271.012,338.089 273.71,337.487 284.104,335.63 294.497,334.257 304.891,333.349 315.284,332.883 
 325.677,332.833 336.071,333.169 346.464,333.86 356.857,334.87 367.251,336.163 377.644,337.697 379.984,338.089 388.038,339.222 398.431,340.814 408.824,342.49 
 419.218,344.202 423.834,344.958 429.611,345.764 440.004,347.158 450.398,348.442 460.791,349.561 471.185,350.457 481.578,351.066 491.971,351.323 502.365,351.155 
 512.758,350.486 523.151,349.235 533.545,347.314 542.668,344.958 543.938,344.575 554.332,340.423 558.917,338.089 564.725,334.583 
 "/>
<polyline clip-path="url(#clip1102)" style="stroke:#9e2a62; stroke-width:1; stroke-opacity:1; fill:none" points="
 564.725,343.386 561.634,344.958 554.332,348.126 543.938,351.67 543.328,351.827 533.545,354.008 523.151,355.671 512.758,356.76 502.365,357.348 491.971,357.507 
 481.578,357.301 471.185,356.794 460.791,356.042 450.398,355.1 440.004,354.02 429.611,352.848 420.897,351.827 419.218,351.598 408.824,350.18 398.431,348.799 
 388.038,347.496 377.644,346.311 367.251,345.276 363.334,344.958 356.857,344.335 346.464,343.587 336.071,343.12 325.677,342.963 315.284,343.142 304.891,343.68 
 294.497,344.597 291.602,344.9

In [45]:
xmat = hcat(reshape((Optim.x_trace(res1)),1, length(Optim.x_trace(res1)))...)
n,m= size(xmat)
plot([[xmat[1,i]] for i = 1:m]', [[xmat[2,i]] for i = 1:m]', arrows=true)

LoadError: [91mMethodError: Cannot `convert` an object of type Array{Float64,2} to an object of type Float64
This may have arisen from a call to the constructor Float64(...),
since type constructors fall back to convert methods.[39m

In the figure above, we've colored in local minimizers as black dots. The minimum of the Himmelblau function is 0, as $f(x_i)=0$ for all $x_i$ in the set of black dots in the figure (the minimizers). Let us try finding them all, by starting sufficiently close to each one.

In [6]:
# find [3.0, 2.0]
res1 = optimize(himmelblau, [2.0,1.0], BFGS(), Optim.Options(store_trace=true, extended_trace=true))

Results of Optimization Algorithm
 * Algorithm: BFGS
 * Starting Point: [2.0,1.0]
 * Minimizer: [2.99999999994967,1.9999999999840121]
 * Minimum: 1.141628e-19
 * Iterations: 9
 * Convergence: true
   * |x - x'| ≤ 1.0e-32: false 
     |x - x'| = 1.16e-07 
   * |f(x) - f(x')| ≤ 1.0e-32 |f(x)|: false
     |f(x) - f(x')| = 3.96e+06 |f(x)|
   * |g(x)| ≤ 1.0e-08: true 
     |g(x)| = 3.77e-10 
   * Stopped by an increasing objective: false
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 24
 * Gradient Calls: 24

In [30]:
#find [-2.805118, 3.131312]
res2 = optimize(himmelblau, [-2.0,1.0], BFGS(), Optim.Options(store_trace=true, extended_trace=true))

Results of Optimization Algorithm
 * Algorithm: BFGS
 * Starting Point: [-2.0,1.0]
 * Minimizer: [-2.805118086901757,3.131312518193761]
 * Minimum: 2.104617e-19
 * Iterations: 8
 * Convergence: true
   * |x - x'| ≤ 1.0e-32: false 
     |x - x'| = 1.67e-09 
   * |f(x) - f(x')| ≤ 1.0e-32 |f(x)|: false
     |f(x) - f(x')| = 4.25e+02 |f(x)|
   * |g(x)| ≤ 1.0e-08: true 
     |g(x)| = 1.51e-13 
   * Stopped by an increasing objective: false
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 27
 * Gradient Calls: 27

In [31]:
#find [-3.779310, -3.283186]
res3 = optimize(himmelblau, [-2.0,-1.0], BFGS(), Optim.Options(store_trace=true, extended_trace=true))

Results of Optimization Algorithm
 * Algorithm: BFGS
 * Starting Point: [-2.0,-1.0]
 * Minimizer: [-3.7793102533232252,-3.2831859912053183]
 * Minimum: 3.366663e-19
 * Iterations: 11
 * Convergence: true
   * |x - x'| ≤ 1.0e-32: false 
     |x - x'| = 9.40e-11 
   * |f(x) - f(x')| ≤ 1.0e-32 |f(x)|: false
     |f(x) - f(x')| = 9.76e-07 |f(x)|
   * |g(x)| ≤ 1.0e-08: true 
     |g(x)| = 3.86e-09 
   * Stopped by an increasing objective: false
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 80
 * Gradient Calls: 80

In [32]:
#find [3.584428, -1.848126]
res4 = optimize(himmelblau, [2.0,-1.0], BFGS(), Optim.Options(store_trace=true, extended_trace=true))

Results of Optimization Algorithm
 * Algorithm: BFGS
 * Starting Point: [2.0,-1.0]
 * Minimizer: [3.584428340262866,-1.8481265269167992]
 * Minimum: 2.504723e-19
 * Iterations: 9
 * Convergence: true
   * |x - x'| ≤ 1.0e-32: false 
     |x - x'| = 4.25e-09 
   * |f(x) - f(x')| ≤ 1.0e-32 |f(x)|: false
     |f(x) - f(x')| = 2.51e+03 |f(x)|
   * |g(x)| ≤ 1.0e-08: true 
     |g(x)| = 7.49e-13 
   * Stopped by an increasing objective: false
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 35
 * Gradient Calls: 35

In [38]:
hcat(Optim.x_trace(res3)...)

2×12 Array{Float64,2}:
 -2.0  -2.76755  -3.56127  -3.71432  …  -3.77931  -3.77931  -3.77931
 -1.0  -1.25585  -1.92093  -2.1718      -3.28319  -3.28319  -3.28319

In [42]:
for res in (res1, res2, res3, res4)
    plot!(p, vcat(Optim.x_trace(res)'...), lw=1, c = :red, lt= :path,  arrows=true)
end
p

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -5 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 20 
 
 
 -4 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 
 
<polyline clip-path="url(#clip9702)" style="stroke:#470b69; stroke-width:1; stroke-opacity:1; fill:none" points="
 45.0562,355.526 47.0965,358.696 48.6401,360.52 52.224,363.284 55.808,364.774 59.3919,365.23 62.9758,364.853 66.5597,363.814 70.1437,362.259 73.7276,360.312 
 76.3048,358.696 77.3115,357.955 80.8954,355.051 84.4793,352.028 84.7097,351.827 88.0633,348.371 91.4187,344.958 91.6472,344.681 95.2311,340.396 97.2541,338.089 
 98.815,335.949 102.399,331.348 102.506,331.22 105.983,326.174 107.38,324.351 109.567,320.837 111.948,317.482 113.151,315.359 116.315,310.613 116.735,309.809 
 120.319,304.191 120.684,303.744 123.902,298.643 125.493,296.875 127.486,293.912 131.07,290.799 133.549,290.006 134.654,289.523 136.782,290.006 138.238,290.224 
 141.822,292.216 145.406,295.45 146.565,296.875 148.99,299.047 152.574,303.018 153.13,303.744 156.158,306.765 159.597,310.613 159.742,310.741 163.326,314.217 
 166.506,317.482 166.91,317.816 170.493,320.911 174.077,324.03 174.451,324.351 177.661,326.623 181.245,329.041 184.767,331.22 184.829,331.252 188.413,332.874 
 191.997,334.176 195.581,335.074 199.165,335.474 202.749,335.28 206.333,334.384 209.917,332.672 211.879,331.22 213.501,329.785 217.084,325.323 217.673,324.351 
 220.668,318.311 220.99,317.482 223.126,310.613 224.252,305.952 
 "/>
<polyline clip-path="url(#clip9702)" style="stroke:#63146e; stroke-width:1; stroke-opacity:1; fill:none" points="
 224.252,323.336 223.859,324.351 220.668,331.074 220.58,331.22 217.084,336.024 215.092,338.089 213.501,339.484 209.917,341.727 206.333,343.181 202.749,343.949 
 199.165,344.128 195.581,343.807 191.997,343.07 188.413,341.995 184.829,340.654 181.245,339.115 179.044,338.089 177.661,337.326 174.077,335.263 170.493,333.173 
 167.09,331.22 166.91,331.095 163.326,328.743 159.742,326.552 156.158,324.578 155.676,324.351 152.574,322.556 148.99,320.866 145.406,319.611 141.822,318.845 
 138.238,318.618 134.654,318.979 131.07,319.976 127.486,321.654 123.902,324.058 123.563,324.351 120.319,326.658 116.735,329.823 115.395,331.22 113.151,333.169 
 109.567,336.754 108.373,338.089 105.983,340.334 102.399,344.042 101.573,344.958 98.815,347.545 95.2311,351.121 94.5454,351.827 91.6472,354.372 88.0633,357.6 
 86.8428,358.696 84.4793,360.516 80.8954,363.219 77.6518,365.565 
 "/>
<polyline clip-path="url(#clip9702)" style="stroke:#821f6b; stroke-width:1; stroke-opacity:1; fill:none" points="
 87.3899,365.565 88.0633,365.031 91.6472,362.167 95.2311,359.376 96.1295,358.696 98.815,356.34 102.399,353.366 104.383,351.827 105.983,350.379 109.567,347.4 
 112.849,344.958 113.151,344.695 116.735,341.912 120.319,339.544 122.972,338.089 123.902,337.487 127.486,335.63 131.07,334.257 134.654,333.349 138.238,332.883 
 141.822,332.833 145.406,333.169 148.99,333.86 152.574,334.87 156.158,336.163 159.742,337.697 160.548,338.089 163.326,339.222 166.91,340.814 170.493,342.49 
 174.077,344.202 175.669,344.958 177.661,345.764 181.245,347.158 184.829,348.442 188.413,349.561 191.997,350.457 195.581,351.066 199.165,351.323 202.749,351.155 
 206.333,350.486 209.917,349.235 213.501,347.314 216.647,344.958 217.084,344.575 220.668,340.423 222.25,338.089 224.252,334.583 
 "/>
<polyline clip-path="url(#clip9702)" style="stroke:#9e2a62; stroke-width:1; stroke-opacity:1; fill:none" points="
 224.252,343.386 223.187,344.958 220.668,348.126 217.084,351.67 216.874,351.827 213.501,354.008 209.917,355.671 206.333,356.76 202.749,357.348 199.165,357.507 
 195.581,357.301 191.997,356.794 188.413,356.042 184.829,355.1 181.245,354.02 177.661,352.848 174.657,351.827 174.077,351.598 170.493,350.18 166.91,348.799 
 163.326,347.496 159.742,346.311 156.158,345.276 154.807,344.958 152.574,344.335 148.99,343.587 145.406,343.12 141.822,342.963 138.238,343.142 134.654,343.68 
 131.07,344.597 1

Below, we use an interactive figure to see how these non-linearities come into play. Try keeping $x_2$ fixed at 0.0, and vary $x_1$ to find all the minimizers. Then try to fix $x_1$ at -4.0. Can you trace out all the minimizers by varying $x_2$?

In [84]:
using Interact
@manipulate for xx = slider(-5.0:.5:5.0, value = -2.0, label="x₁"), yy = slider(-5.0:0.5:5.0, value = 0.0, label="x₂")
    _x = [xx, yy]
    contour(-6.0:0.1:6.0, -6.0:0.1:6.0, (x,y)->himmelblau([x, y]), fill=true, color=:deep, legend=false)
    contour!(-6.0:0.1:6.0, -6.0:0.1:6.0, (x,y)->himmelblau([x, y]), fill=false, legend=false)
    res = optimize(himmelblau, _x, BFGS(), Optim.Options(store_trace=true, extended_trace=true))
    xtracemat1 = hcat(Optim.x_trace(res)...)
    plot!(xtracemat[1, :], xtracemat[2, :], mc = :white)
    scatter!(xtracemat[1, :], xtracemat[2, :], mc = :white)
   #
    scatter!([0.], [0.], mc=:red, msc=:red)
end

Interact.Slider{Float64}(83: "input-31" = -2.0 Float64 , "x₁", -2.0, -5.0:0.5:5.0, "horizontal", true, ".3f", true)

Interact.Slider{Float64}(85: "input-32" = 0.0 Float64 , "x₂", 0.0, -5.0:0.5:5.0, "horizontal", true, ".3f", true)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -5.0 
 
 
 -2.5 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 -5.0 
 
 
 -2.5 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 
 
 
<polyline clip-path="url(#clip02)" style="stroke:#63146e; stroke-width:1; stroke-opacity:1; fill:none" points="
 54.4598,343.403 56.3977,345.48 58.7142,347.687 59.4722,348.342 62.9687,351.048 63.1921,351.203 67.2232,353.738 67.8047,354.065 71.4777,355.939 73.6672,356.927 
 75.7322,357.778 79.9867,359.315 81.4909,359.788 84.2412,360.582 88.4957,361.634 92.7502,362.52 93.5031,362.65 97.0047,363.208 101.259,363.754 105.514,364.176 
 109.768,364.482 114.023,364.681 118.277,364.781 122.532,364.791 126.786,364.717 131.041,364.566 135.295,364.346 139.55,364.061 143.804,363.719 148.059,363.325 
 152.313,362.884 154.374,362.65 156.568,362.385 160.822,361.832 165.077,361.245 169.331,360.628 173.586,359.987 174.859,359.788 177.84,359.294 182.095,358.571 
 186.349,357.834 190.603,357.089 191.516,356.927 194.858,356.297 199.112,355.492 203.367,354.688 206.676,354.065 207.621,353.876 211.876,353.029 216.13,352.193 
 220.385,351.371 221.269,351.203 224.639,350.521 228.894,349.68 233.148,348.861 235.923,348.342 237.403,348.047 241.657,347.225 245.912,346.434 250.166,345.674 
 251.3,345.48 254.421,344.91 258.675,344.17 262.93,343.469 267.184,342.807 268.467,342.618 271.439,342.154 275.693,341.532 279.948,340.954 284.202,340.421 
 288.457,339.934 290.155,339.757 292.711,339.472 296.966,339.046 301.22,338.669 305.475,338.339 309.729,338.056 313.984,337.819 318.238,337.628 322.493,337.482 
 326.747,337.379 331.002,337.319 335.256,337.299 339.511,337.319 343.765,337.378 348.02,337.472 352.274,337.601 356.529,337.763 360.783,337.955 365.038,338.176 
 369.292,338.424 373.547,338.695 377.801,338.988 382.056,339.299 386.31,339.627 387.92,339.757 390.565,339.955 394.819,340.285 399.074,340.622 403.328,340.963 
 407.583,341.304 411.837,341.644 416.092,341.978 420.346,342.304 424.601,342.616 424.626,342.618 428.855,342.895 433.11,343.155 437.364,343.393 441.619,343.604 
 445.873,343.785 450.128,343.933 454.382,344.042 458.636,344.108 462.891,344.128 467.145,344.096 471.4,344.008 475.654,343.859 479.909,343.644 484.163,343.358 
 488.418,342.995 492.029,342.618 492.672,342.547 496.927,341.981 501.181,341.316 505.436,340.546 509.243,339.757 509.69,339.658 513.945,338.59 518.199,337.39 
 519.767,336.895 522.454,335.989 526.708,334.395 527.582,334.033 530.963,332.533 533.752,331.172 535.217,330.404 538.863,328.31 539.472,327.933 543.153,325.448 
 543.726,325.031 546.809,322.587 547.981,321.581 549.974,319.725 552.235,317.438 552.761,316.863 555.163,314.002 556.49,312.272 557.295,311.14 559.149,308.278 
 560.744,305.563 560.824,305.417 562.234,302.555 563.499,299.693 564.627,296.832 564.999,295.767 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#821f6b; stroke-width:1; stroke-opacity:1; fill:none" points="
 564.999,319.776 562.603,322.587 560.744,324.599 559.899,325.448 556.825,328.31 556.49,328.599 553.277,331.172 552.235,331.948 549.208,334.033 547.981,334.822 
 544.481,336.895 543.726,337.313 539.472,339.479 538.876,339.757 535.217,341.356 532.053,342.618 530.963,343.027 526.708,344.476 523.454,345.48 522.454,345.77 
 518.199,346.882 513.945,347.877 511.717,348.342 509.69,348.739 505.436,349.476 501.181,350.121 496.927,350.679 492.672,351.155 492.159,351.203 488.418,351.535 
 484.163,351.844 479.909,352.09 475.654,352.278 471.4,352.41 467.145,352.493 462.891,352.529 458.636,352.523 454.382,352.478 450.128,352.398 445.873,352.287 
 441.619,352.148 437.364,351.985 433.11,351.8 428.855,351.598 424.601,351.38 421.32,351.203 420.346,351.147 416.092,350.895 411.837,350.635 407.583,350.372 
 403.328,350.107 399.074,349.845 394.819,349.586 390.565,349.335 386.31,349.092 382.056,348.861 377.801,348.644 373.547,348.442 371.211,348.342 369.292,348.253 
 365.038,348.079 360.783,347.927 356.529,347.799 352.274,347.699 348.02,347.626 34

The lesson is clear. Non-linear functions can be a tricky thing to minimize, and the Himmelblau function is not even that hard a case! This is a very important thing to keep in mind when trying out the different methods in Optim, and wondering about inconconsistencies between runs, starting points, methods, and more.

# Syntax
To optimize (minimize) a function, you have to provide, *at the very least*, a function to calculate the objective and a starting point.
```julia
optimize(objective args..., initial_x, [method], [options])
```
some examples...

In [88]:
# Supply objective and initial point
# equivalent to:
# optimize(himmelblau, [2.0, -1.0], NelderMead(), Optim.Options())
optimize(himmelblau, [2.0,-1.0])

Results of Optimization Algorithm
 * Algorithm: Nelder-Mead
 * Starting Point: [2.0,-1.0]
 * Minimizer: [3.5844298648286093,-1.8481263812232966]
 * Minimum: 1.236197e-10
 * Iterations: 34
 * Convergence: true
   *  √(Σ(yᵢ-ȳ)²)/n < 1.0e-08: true
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 71

In [91]:
# Supply objective, initial point and method
# equivalent to:
# optimize(himmelblau, [2.0, -1.0], BFGS(), Optim.Options()
# Notice, gradient is approximated by finite differences.
optimize(himmelblau, [2.0,-1.0], BFGS())

Results of Optimization Algorithm
 * Algorithm: BFGS
 * Starting Point: [2.0,-1.0]
 * Minimizer: [3.584428340262866,-1.8481265269167992]
 * Minimum: 2.504723e-19
 * Iterations: 9
 * Convergence: true
   * |x - x'| < 1.0e-32: false 
     |x - x'| = 4.25e-09 
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: false
     |f(x) - f(x')| / |f(x)| = NaN 
   * |g(x)| < 1.0e-08: true 
     |g(x)| = 7.49e-13 
   * stopped by an increasing objective: false
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 35
 * Gradient Calls: 35

In [92]:
# Supply objective, gradient and initial point
# equivalent to:
# optimize(himmelblau, himmelblau_gradient!, [2.0, -1.0], BFGS(), Optim.Options())
optimize(himmelblau, himmelblau_gradient!, [2.0,-1.0])

Results of Optimization Algorithm
 * Algorithm: L-BFGS
 * Starting Point: [2.0,-1.0]
 * Minimizer: [3.5844283403304593,-1.8481265269644178]
 * Minimum: 6.138363e-26
 * Iterations: 8
 * Convergence: true
   * |x - x'| < 1.0e-32: false 
     |x - x'| = 2.25e-08 
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: false
     |f(x) - f(x')| / |f(x)| = 1.25e+11 
   * |g(x)| < 1.0e-08: true 
     |g(x)| = 3.51e-12 
   * stopped by an increasing objective: false
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 34
 * Gradient Calls: 34

In [96]:
# Supply objective, gradient, initial point and method
# equivalent to:
# optimize(himmelblau, [2.0, -1.0], BFGS(), Optim.Options())
# Notice, hessian is approximated by finite differences.
optimize(himmelblau, himmelblau_gradient!, [2.0,-1.0],  Newton())

Results of Optimization Algorithm
 * Algorithm: Newton's Method
 * Starting Point: [2.0,-1.0]
 * Minimizer: [3.5844283403304913,-1.8481265269644038]
 * Minimum: 1.262177e-29
 * Iterations: 6
 * Convergence: true
   * |x - x'| < 1.0e-32: false 
     |x - x'| = 6.81e-08 
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: false
     |f(x) - f(x')| / |f(x)| = NaN 
   * |g(x)| < 1.0e-08: true 
     |g(x)| = 5.68e-14 
   * stopped by an increasing objective: false
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 31
 * Gradient Calls: 31
 * Hessian Calls: 6

In [97]:
# Supply objective, gradient, initial point
# equivalent to:
# optimize(himmelblau, himmelblau_gradient!, himmelblau_hessian!,  [2.0,-1.0],  Newton()), Optim.Options())
optimize(himmelblau, himmelblau_gradient!, himmelblau_hessian!,  [2.0,-1.0],  Newton())

Results of Optimization Algorithm
 * Algorithm: Newton's Method
 * Starting Point: [2.0,-1.0]
 * Minimizer: [3.5844283403304913,-1.848126526964404]
 * Minimum: 1.577722e-29
 * Iterations: 6
 * Convergence: true
   * |x - x'| < 1.0e-32: false 
     |x - x'| = 6.83e-08 
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: false
     |f(x) - f(x')| / |f(x)| = NaN 
   * |g(x)| < 1.0e-08: true 
     |g(x)| = 5.33e-14 
   * stopped by an increasing objective: false
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 31
 * Gradient Calls: 31
 * Hessian Calls: 6